<a href="https://colab.research.google.com/github/Shankjbs571/Machine-Learning-Techniques/blob/main/DataPreprocessing_W1_GA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sklearn
import numpy as np

# from sklearn.preprocessing import
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer


In [2]:
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
sklearn.__version__

'1.2.2'

In [77]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/DataPreprocessingGraded_dataset (1).csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      748 non-null    object 
 1   V2      748 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB


In [78]:
dataset.eq('?').sum()

V1        5
V2        5
V3        0
V4        0
V5        0
Target    0
dtype: int64

In [79]:
dataset.replace('?',np.nan,inplace=True)
# dataset.replace('NEGATIVE',1,inplace=True)
print(dataset.head())
print(dataset.info())
dataset['V5'].unique()

    V1    V2       V3    V4        V5 Target
0  2.0  50.0  12500.0  98.0  NEGATIVE    YES
1  0.0  13.0   3250.0  28.0  NEGATIVE    YES
2  NaN   NaN   4000.0  35.0  NEGATIVE    YES
3  NaN  20.0   5000.0  45.0  NEGATIVE    YES
4  1.0  24.0   6000.0  77.0  NEGATIVE     NO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      743 non-null    object 
 1   V2      743 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB
None


array(['NEGATIVE'], dtype=object)

In [80]:
oe=OrdinalEncoder()
encoded_columns = oe.fit_transform(dataset[['V5', 'Target']])
print(type(encoded_columns))
# # Convert the encoded columns to a DataFrame
encoded_df = pd.DataFrame(encoded_columns, columns=oe.get_feature_names_out(['V5', 'Target']))
print(encoded_df)

si = SimpleImputer(strategy='mean')
imputed_columns = si.fit_transform(dataset[['V1','V2']])
print(type(imputed_columns))
imputed_df = pd.DataFrame(imputed_columns, columns=si.get_feature_names_out(['V1', 'V2']))
print(imputed_df)


<class 'numpy.ndarray'>
      V5  Target
0    0.0     1.0
1    0.0     1.0
2    0.0     1.0
3    0.0     1.0
4    0.0     0.0
..   ...     ...
743  0.0     0.0
744  0.0     0.0
745  0.0     0.0
746  0.0     0.0
747  0.0     0.0

[748 rows x 2 columns]
<class 'numpy.ndarray'>
            V1         V2
0     2.000000  50.000000
1     0.000000  13.000000
2     9.562584   5.464334
3     9.562584  20.000000
4     1.000000  24.000000
..         ...        ...
743  23.000000   2.000000
744  21.000000   2.000000
745  23.000000   3.000000
746  39.000000   1.000000
747  72.000000   1.000000

[748 rows x 2 columns]


In [81]:
# final_df = pd.concat([dataset, encoded_df], axis=1)
# # Drop the original non-numeric columns
print(dataset.info())

dataset.drop(['V1','V2','V5', 'Target'], axis=1, inplace=True)
print(dataset.info())
concat_dataset = pd.concat([dataset, encoded_df,imputed_df], axis=1)
print(concat_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      743 non-null    object 
 1   V2      743 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V3      748 non-null    float64
 1   V4      748 non-null    float64
dtypes: float64(2)
memory usage: 11.8 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V3      748 non-null    float64
 1   V4      748 non-null    

In [82]:
concat_dataset.head()

,V3,V4,V5,Target,V1,V2
0,12500.0,98.0,0.0,1.0,2.000000,50.000000
1,3250.0,28.0,0.0,1.0,0.000000,13.000000
2,4000.0,35.0,0.0,1.0,9.562584,5.464334
3,5000.0,45.0,0.0,1.0,9.562584,20.000000
4,6000.0,77.0,0.0,0.0,1.000000,24.000000


In [83]:
# Extract feature columns (excluding the target column)
feature_columns = [col for col in concat_dataset.columns if col != 'Target']

# Apply VarianceThreshold to feature columns only
selector = VarianceThreshold(threshold=0.1)
selected_features = selector.fit_transform(concat_dataset[feature_columns])

# Convert the selected features to a DataFrame
selected_df = pd.DataFrame(selected_features, columns=concat_dataset[feature_columns].columns[selector.get_support()])
print(selected_df.info())
print(selected_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V3      748 non-null    float64
 1   V4      748 non-null    float64
 2   V1      748 non-null    float64
 3   V2      748 non-null    float64
dtypes: float64(4)
memory usage: 23.5 KB
None
        V3    V4        V1         V2
0  12500.0  98.0  2.000000  50.000000
1   3250.0  28.0  0.000000  13.000000
2   4000.0  35.0  9.562584   5.464334
3   5000.0  45.0  9.562584  20.000000
4   6000.0  77.0  1.000000  24.000000


In [87]:
# Create the RFE model and specify the number of features to select (in this case, 2)
rfe = RFE(LogisticRegression(), n_features_to_select=2)

# Fit the RFE model to the data
# rfe.fit_transform(piped_dataset[:4],piped_dataset[5])
selected_features = pd.DataFrame(rfe.fit_transform(concat_dataset[feature_columns],concat_dataset['Target']))
print(selected_features.head())
print()
# Get the indices of the selected features
# selected_features_indices = rfe.get_support(indices=True)

# Get the names of the selected features
# selected_features_names = piped_dataset.columns[selected_features_indices]

# print("Selected features:", selected_features_names)

          0          1
0  2.000000  50.000000
1  0.000000  13.000000
2  9.562584   5.464334
3  9.562584  20.000000
4  1.000000  24.000000


In [88]:
from sklearn.feature_selection import SequentialFeatureSelector
? SequentialFeatureSelector

In [91]:
# Create a Sequential Forward Selection object
sfs = SequentialFeatureSelector(LogisticRegression(),
          n_features_to_select=2,
          direction='forward'
          )
# Get the indices of the selected features
sfs.fit(concat_dataset[feature_columns],concat_dataset['Target'])
selected_indices = sfs.get_support(indices=True)

print("Indices of the two most important features:", selected_indices)
selected_sfs_features = pd.DataFrame(sfs.fit_transform(concat_dataset[feature_columns],concat_dataset['Target']))
print(selected_sfs_features.head())

Indices of the two most important features: [1 4]
      0          1
0  98.0  50.000000
1  28.0  13.000000
2  35.0   5.464334
3  45.0  20.000000
4  77.0  24.000000


In [92]:
# Create a Sequential Forward Selection object
sfs = SequentialFeatureSelector(LogisticRegression(),
          n_features_to_select=2,
          direction='backward'
          )
# Get the indices of the selected features
sfs.fit(concat_dataset[feature_columns],concat_dataset['Target'])
selected_indices = sfs.get_support(indices=True)

print("Indices of the two most important features:", selected_indices)
selected_sfs_features = pd.DataFrame(sfs.fit_transform(concat_dataset[feature_columns],concat_dataset['Target']))
print(selected_sfs_features.head())

Indices of the two most important features: [0 1]
         0     1
0  12500.0  98.0
1   3250.0  28.0
2   4000.0  35.0
3   5000.0  45.0
4   6000.0  77.0
